In [1]:
#%pip install pandas
#%pip install numpy
#%pip install langchain
#%pip install langchain-community
#%pip install langchain-groq
#%pip install sentence-transformers
#%pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os 
import json

#define groq key 
groq_key = os.environ['groqkey']


In [2]:
#model variables
llama_70 = 'llama3-70b-8192'
llama_31 = 'llama-3.1-70b-versatile'
mixtral = "mixtral-8x7b-32768"
gemma_2 = 'gemma2-9b-it'
llama_tool_70 = 'llama3-groq-70b-8192-tool-use-preview'

gemma = ChatGroq(
    temperature=0,
    model=gemma_2,
    api_key=groq_key 
)

mixtral = ChatGroq(
    temperature=0,
    model=mixtral,
    api_key=groq_key 
)

llama3 = ChatGroq(
    temperature=0,
    model=llama_70,
    api_key=groq_key 
)

llama_tool = ChatGroq(
    temperature=0,
    model=llama_70,
    api_key=groq_key 
)

llama_3_1 = ChatGroq(
    temperature=0,
    model=llama_31,
    api_key=groq_key 
)

In [3]:
system = "You are a knowledgeable medical provider who specializes in medication management."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llama_3_1
chain.invoke({"text": "Explain what should be evaluated for PPI deprescribing for a patient at discharge."})

AIMessage(content="When considering PPI (Proton Pump Inhibitor) deprescribing for a patient at discharge, several factors should be evaluated to ensure safe and effective discontinuation. Here are the key considerations:\n\n1. **Indication for PPI use**: Review the original reason for starting the PPI. If the indication was for a short-term condition, such as gastroesophageal reflux disease (GERD) or peptic ulcer disease, and the condition has resolved, deprescribing may be appropriate.\n2. **Duration of PPI use**: Assess the length of time the patient has been taking the PPI. Long-term use (more than 8-12 weeks) increases the risk of rebound acid hypersecretion and may make deprescribing more challenging.\n3. **Dose and frequency**: Evaluate the current PPI dose and frequency. Patients on higher doses or more frequent dosing may require a more gradual taper to minimize rebound symptoms.\n4. **Symptom control**: Assess the patient's current symptoms and whether they are well-controlled

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
base_embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
text_splitter = RecursiveCharacterTextSplitter()

C:\Users\micha\AppData\Local\Temp\ipykernel_9396\1026861699.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  base_embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
f:\LangChain\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
base_path = "F:/LangChain/data/"
noteConcepts = pd.read_csv(base_path + 'noteConcepts.txt', sep="|")
encounters = pd.read_csv(base_path + 'encounters.txt', sep="|")
orders = pd.read_csv(base_path + 'orders.txt', sep="|")
hospitalAcquiredDx = pd.read_csv(base_path + 'hospitalAcquiredDx.txt', sep="|")
noteText = pd.read_csv(base_path + 'noteText.txt', sep="|")
presentOnAdmitDx = pd.read_csv(base_path + 'presentOnAdmitDx.txt', sep="|")
#select random EncounterKey
encounterKey = encounters.sample(1)['EncounterKey'].values[0]

In [6]:
#filter all dfs to the selected EncounterKey
noteConcepts = noteConcepts[noteConcepts['EncounterKey'] == encounterKey]
orders = orders[orders['EncounterKey'] == encounterKey]
hospitalAcquiredDx = hospitalAcquiredDx[hospitalAcquiredDx['EncounterKey'] == encounterKey]
noteText = noteText[noteText['EncounterKey'] == encounterKey]
presentOnAdmitDx = presentOnAdmitDx[presentOnAdmitDx['EncounterKey'] == encounterKey]
encounters = encounters[encounters['EncounterKey'] == encounterKey]

In [7]:
#change encounters to a json object
encounters = encounters.to_json(orient="records")
print(encounters)

[{"EncounterKey":"D5572323A153F5","PatientKey":"D6A6FA9D7F0784","Sex":"Female","Age_y":11,"BirthDate":"2012-04-27","PtAdmitDate":"2023-08-02","PtDischargeDate":"2023-09-04","DRG":"MAJOR PANCREAS, LIVER AND SHUNT PROCEDURES","FinancialClass":"Medi-Cal Standard","AdmissionOrigin":"Direct Admission","AdmissionSource":"Physician Referral","AdmissionType":"Routine\/Elective","PrimaryDx":"Other chronic pancreatitis  (CMS code)","PresentOnAdmissionDiagnosisComboKey":-1,"HospitalAcquiredDiagnosisComboKey":-1,"DischargeDisposition":"Home or Self Care","DischargePatientClass":"Inpatient"}]


In [8]:
def extract_json_from_content(content):
    # Find the JSON part within the content
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_str = content[start_index:end_index]

    # Parse the JSON string
    parsed_json = json.loads(json_str)
    
    return parsed_json

In [9]:
system = "You are a knowledgeable medical provider who specializes in medication management."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llama_3_1
response = chain.invoke({"text": f"""Based on the information from this encounter JSON information: {encounters}, does the patient have any of the following:
              1. Mild to moderate esoophagitis
              2. GERD 
              3. Peptic Ulcer Disease
              4. Upper GI symptoms
              5. ICU Stress Ulcer Prophylaxis
              6. Barrett's Esophagus
              7. Chronic NSAID use with bleeding risk
              8. Severe esophagitis
              9. Documented history of bleeding GI ulcer
              10. Explain the reasoning for your answer
            Return the answer for each of these as a formatted JSON object with the key being the condition and the value being a boolean value for the first 9.  For the final question, return a string with the reasoning for your answer."""})
print(response)

content='Based on the provided JSON information, here\'s the answer for each condition:\n\n```json\n{\n  "Mild to moderate esophagitis": false,\n  "GERD": false,\n  "Peptic Ulcer Disease": false,\n  "Upper GI symptoms": false,\n  "ICU Stress Ulcer Prophylaxis": false,\n  "Barrett\'s Esophagus": false,\n  "Chronic NSAID use with bleeding risk": false,\n  "Severe esophagitis": false,\n  "Documented history of bleeding GI ulcer": false,\n  "Reasoning": "The provided JSON information does not explicitly mention any of the listed conditions. The primary diagnosis is \'Other chronic pancreatitis\', which is related to the pancreas, not the esophagus or stomach. The DRG (Diagnosis-Related Group) is \'MAJOR PANCREAS, LIVER AND SHUNT PROCEDURES\', which also suggests that the patient\'s condition is related to the pancreas and liver, rather than the esophagus or stomach. Therefore, based on the available information, it is not possible to confirm the presence of any of the listed conditions."\n

In [10]:
extract_json_from_content(response.content)

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': False,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': False,
 'Reasoning': "The provided JSON information does not explicitly mention any of the listed conditions. The primary diagnosis is 'Other chronic pancreatitis', which is related to the pancreas, not the esophagus or stomach. The DRG (Diagnosis-Related Group) is 'MAJOR PANCREAS, LIVER AND SHUNT PROCEDURES', which also suggests that the patient's condition is related to the pancreas and liver, rather than the esophagus or stomach. Therefore, based on the available information, it is not possible to confirm the presence of any of the listed conditions."}